In [1]:
# importing the libraries
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import stats

from sklearn.preprocessing import LabelEncoder

In [2]:
# defining few parameters
root_dir = os.path.dirname(os.path.abspath(os.getcwd())) # setting up root directory
warnings.filterwarnings("ignore")
sns.set_style("darkgrid")                                # graph style
plt.rcParams["figure.figsize"] = (12, 8)                 # graph size

In [3]:
# defining the paths
trainpath = os.path.join(root_dir, "data", "train.csv")
item_data_path = os.path.join(root_dir, "data", "item_data.csv")
view_log_path = os.path.join(root_dir, "data", "view_log.csv")
testpath = os.path.join(root_dir, "data", "test.csv")

# importing the datasets
train_df = pd.read_csv(trainpath)
item_df = pd.read_csv(item_data_path)
view_df = pd.read_csv(view_log_path)
test_df = pd.read_csv(testpath)

In [4]:
train_df.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


### Column descriptions for train_df - 

| Column Name | Column Description |
|----|----|
| impression_id | Ad impression ID 1 (unique). Shouldn't be fed to the model |
| impression_time | time of impression at the partner site |
| user_id | user identification |
| app code | Application Code for a partner website where the ad was shown |
| os_version | version of operating system |
| is_4G | 1 (Using 4G), 0 (No 4G) |
| is_click | **target**; 0 (No Click), 1 (Click) |

In [5]:
item_df.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


Variables are quite self-explanatory. But can't be mapped to the main column as there is not shared column. 
- item_id is unique in this dataset

In [6]:
view_df.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


### Column Description for view_df -
| Column Name | Column Description |
|----|----|
| server_time | Timestamp of the log |
| device_type | Device type of the user |
| browser_id | Browser session id |
| user_id | can be used to map this table to traindf |
| item_id | can be used to map this table to item_data_df |

We have two attributes that captures the time, ``server_time`` in ``view_df`` and ``impression_time`` in ``train_df``. These attributes can help us to validate whether the dataset have a time component influencing the label or not.

For this, we will first convert them into datetime format and then sort them.

In [7]:
# 01. converting date columns into datetime format

view_df["server_time"] = pd.to_datetime(view_df["server_time"])
train_df["impression_time"] = pd.to_datetime(train_df["impression_time"])

view_df = view_df.sort_values("server_time").reset_index(drop = True)
train_df = train_df.sort_values("impression_time").reset_index(drop = True)

In [8]:
%%time
# 02. Datetime features
# the following snippet gives the mean impression time difference for a particular user.
# np.timedelta helps us to perform arithmentic on two timedeltas. The first argument defines a number and second defines the unit of time. 
# Here, that unit is seconds

train_df["diff_time_mean"] = train_df["user_id"].map(
    train_df.groupby("user_id")["impression_time"].apply(lambda x: np.nanmean(x.diff() / np.timedelta64(1, "s"))).to_dict()
)

# similarly performing some other functions
train_df["diff_time_max"] = train_df["user_id"].map(
    train_df.groupby("user_id")["impression_time"].apply(lambda x: np.nanmax(x.diff() / np.timedelta64(1, "s"))).to_dict()
)
train_df["diff_time_min"] = train_df["user_id"].map(
    train_df.groupby("user_id")["impression_time"].apply(lambda x: np.nanmin(x.diff() / np.timedelta64(1, "s"))).to_dict()
)

CPU times: user 2min 18s, sys: 233 ms, total: 2min 18s
Wall time: 2min 19s


In [9]:
train_df.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,diff_time_mean,diff_time_max,diff_time_min
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,74670.000000,148200.0,1140.0
1,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,89464,129,intermediate,0,0,101901.818182,347520.0,180.0
2,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,58442,127,latest,0,0,68812.727273,165720.0,8400.0
3,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,4238,371,latest,0,0,540.000000,540.0,540.0
4,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,45745.882353,167340.0,360.0


In [10]:
# 03. encoding os_version

encoder_1 = LabelEncoder()
os_encoder = encoder_1.fit(train_df["os_version"])

# serializing and dumping the encoder for future use (for encoding the testset)
with open(os.path.join(root_dir, "models", "os_encoder"), 'wb') as f:
    pickle.dump(os_encoder, f)
    
# transforming
train_df["os_version"] = os_encoder.transform(train_df["os_version"])

############################################################################
# if loading then run following snippet                                    #
"""                                                                        #
with open(os.path.join(root_dir, "models", "os_encoder"), 'rb') as handle: #
    b = pickle.load(handle)                                                #
"""                                                                        #
############################################################################

'                                                                        #\nwith open(os.path.join(root_dir, "models", "os_encoder"), \'rb\') as handle: #\n    b = pickle.load(handle)                                                #\n'

In [11]:
# 04. count unique apps used by the user
train_df["count_unique_app"] = train_df["user_id"].map(
    train_df.groupby("user_id")["app_code"].apply(lambda x: x.nunique()).to_dict()
)

train_df.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,diff_time_mean,diff_time_max,diff_time_min,count_unique_app
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,2,0,0,74670.000000,148200.0,1140.0,1
1,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,89464,129,0,0,0,101901.818182,347520.0,180.0,1
2,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,58442,127,1,0,0,68812.727273,165720.0,8400.0,1
3,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,4238,371,1,0,0,540.000000,540.0,540.0,1
4,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,1,1,1,45745.882353,167340.0,360.0,2


In [12]:
# 05. fetch hour and minute from impression time
train_df["hour"] = train_df["impression_time"].dt.hour
train_df["minute"] = train_df["impression_time"].dt.minute

In [13]:
# 06. Calculating the counts of user_id and app_code

for i in ["app_code", "user_id"]:
    train_df["vc_" + i] = train_df[i].map(train_df[i].value_counts().to_dict())
    
    # saving the dictionary for future use
    dict_name = "vc_" + i
    with open(os.path.join(root_dir, "models", dict_name), 'wb') as f:
        pickle.dump(os_encoder, f)

In [14]:
# merge the view_df and item_df on item_id
view_aggdf = view_df.merge(item_df, how = "left", on = "item_id")
view_aggdf["server_time"] = pd.to_datetime(view_aggdf["server_time"])
print("shape of view_df = {}".format(view_df.shape))
print("shape of view_aggdf = {}".format(view_aggdf.shape))

shape of view_df = (3118622, 5)
shape of view_aggdf = (3118622, 10)


In [15]:
# 07. count unique user for an app

train_df["app_code_count_unique_user"] = train_df["app_code"].map(
    train_df.groupby("app_code")["user_id"].apply(lambda x: x.nunique()).to_dict()
)

In [16]:
# 08. Checking how many times user has clicked the ad and what was the last time he had an impression

click_count_user_mean = []  # captures mean of how many times user has clicked in the past
time_elapsed_user = []      # captures the time elapsed since last impression by a user

for uid, imp_time in tqdm(zip(train_df["user_id"].values, train_df["impression_time"].values)):
    temp_df = train_df[((train_df["impression_time"] < imp_time) & (train_df["user_id"] == uid))]
    click_count_user_mean += [-1] if (temp_df["is_click"].mean() == np.nan) else [temp_df["is_click"].mean()]
    if temp_df.shape[0]>0:
        time_elapsed_user += [(imp_time - temp_df["impression_time"].values[-1]) / np.timedelta64(1, "s")] 
    else:
        time_elapsed_user += [-1]

train_df["click_count_user_mean"] = click_count_user_mean
train_df["click_count_user_mean"] = train_df["click_count_user_mean"].fillna(-1)
train_df["time_elapsed_user"] = time_elapsed_user

237609it [09:53, 400.53it/s]


In [17]:
train_df[train_df["user_id"] == 85249]

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,diff_time_mean,diff_time_max,diff_time_min,count_unique_app,hour,minute,vc_app_code,vc_user_id,app_code_count_unique_user,click_count_user_mean,time_elapsed_user
63620,d0ca712ec64f54c9c36d6391cbecfad4,2018-11-23 15:18:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,15,18,295,6,62,-1.0,-1.0
63884,c1a6f8d6e0ca095741fa2d223743fb47,2018-11-23 15:57:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,15,57,295,6,62,0.0,2340.0
70216,fb3c0b3c15a808f75f15bc1a9ad18f3b,2018-11-24 14:23:00,85249,283,1,1,0,260844.0,1177020.0,2340.0,2,14,23,4622,6,900,0.0,80760.0
71009,79fa7cd8a14cdf2ffb14a2b398e33d36,2018-11-24 16:58:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,16,58,295,6,62,0.0,9300.0
205928,e32df0a5f806f41d628632cf70951c00,2018-12-08 07:55:00,85249,283,1,0,0,260844.0,1177020.0,2340.0,2,7,55,4622,6,900,0.0,1177020.0
208600,607fb3a759fb2d34ad3efc2a4dea6cdb,2018-12-08 17:35:00,85249,421,1,0,0,260844.0,1177020.0,2340.0,2,17,35,295,6,62,0.0,34800.0


In [18]:
%%time
# 09. Checking how many times user has clicked the ad from  A PARTICULAR APP and what was the last time he had an impression

click_count_app_mean = []   # captures mean of how many times user has clicked in the past from a particular APP
time_elapsed_app = []       # captures the time elapsed since last impression by a user from a particular APP

for app_code, imp_time in zip(train_df["app_code"].values, train_df["impression_time"].values):
    temp_df = train_df[((train_df["impression_time"] < imp_time) & (train_df["app_code"] == app_code))]
    click_count_app_mean += [-1] if (temp_df["is_click"].mean() == np.nan) else [temp_df["is_click"].mean()]
    if temp_df.shape[0]>0:
        time_elapsed_app += [(imp_time - temp_df["impression_time"].values[-1]) / np.timedelta64(1, "s")] 
    else:
        time_elapsed_app += [-1]
        
train_df["click_count_app_mean"] = click_count_app_mean
train_df["click_count_app_mean"] = train_df["click_count_app_mean"].fillna(-1)
train_df["time_elapsed_app"] = time_elapsed_app

CPU times: user 13min 52s, sys: 510 ms, total: 13min 53s
Wall time: 13min 53s


In [19]:
train_df[train_df["user_id"] == 85249]

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,diff_time_mean,diff_time_max,diff_time_min,count_unique_app,hour,minute,vc_app_code,vc_user_id,app_code_count_unique_user,click_count_user_mean,time_elapsed_user,click_count_app_mean,time_elapsed_app
63620,d0ca712ec64f54c9c36d6391cbecfad4,2018-11-23 15:18:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,15,18,295,6,62,-1.0,-1.0,0.102804,143160.0
63884,c1a6f8d6e0ca095741fa2d223743fb47,2018-11-23 15:57:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,15,57,295,6,62,0.0,2340.0,0.101852,2340.0
70216,fb3c0b3c15a808f75f15bc1a9ad18f3b,2018-11-24 14:23:00,85249,283,1,1,0,260844.0,1177020.0,2340.0,2,14,23,4622,6,900,0.0,80760.0,0.056015,6600.0
71009,79fa7cd8a14cdf2ffb14a2b398e33d36,2018-11-24 16:58:00,85249,421,1,1,0,260844.0,1177020.0,2340.0,2,16,58,295,6,62,0.0,9300.0,0.096491,12420.0
205928,e32df0a5f806f41d628632cf70951c00,2018-12-08 07:55:00,85249,283,1,0,0,260844.0,1177020.0,2340.0,2,7,55,4622,6,900,0.0,1177020.0,0.055116,660.0
208600,607fb3a759fb2d34ad3efc2a4dea6cdb,2018-12-08 17:35:00,85249,421,1,0,0,260844.0,1177020.0,2340.0,2,17,35,295,6,62,0.0,34800.0,0.081897,8520.0


In [3]:
# saving as dataset to avoid running of above code all over again
#train_df.to_csv(os.path.join(root_dir, "data", "processed_data", "train_df.csv"), index = False)
#view_aggdf.to_csv(os.path.join(root_dir, "data", "processed_data", "view_aggdf.csv"), index = False)

# importing the dataset
train_df = pd.read_csv(os.path.join(root_dir, "data", "processed_data", "train_df.csv"))
view_aggdf = pd.read_csv(os.path.join(root_dir, "data", "processed_data", "view_aggdf.csv"))

# converting to datetime
view_aggdf["server_time"] = pd.to_datetime(view_aggdf["server_time"])

The transformations on the **``train_df``** are complete now. In the raw ``train_df``, we had 6 predictors and after few feature engineering steps, now we have 19 features. A brief description of all of these columns is as below:

| Column Name| Column Description|
|-----|-----|
|impression_id | Ad impression ID 1 (unique). Shouldn't be fed to the model |
|impression_time | time of impression at the partner site |
|user_id | user identification |
|app code | Application Code for a partner website where the ad was shown |
|os_version | version of operating system |
|is_4G | 1 (Using 4G), 0 (No 4G) |
|is_click | **target**; 0 (No Click), 1 (Click) |
|diff_time_mean | mean first order difference of the impression time. Given by a[n+1] - a[n] |
|diff_time_max | max first order difference of the impression time |
|diff_time_min | min first order difference of the impression time |
|count_unique_app | unique apps the user have logged in from |
|hour | derived from impression_time |
|minute | derived from impression_time |
|vc_app_code | value count for a particular app |
|vc_user_id | value count for a particular user |
|app_code_count_unique_user | unique user for a particular app |
|click_count_user_mean | captures mean of how many times user has clicked in the past |
|time_elapsed_user | captures the time elapsed since last impression by a user |
|click_count_app_mean | captures mean of how many times user has clicked in the past from a particular APP |
|time_elapsed_app | captures the time elapsed since last impression by a user from a particular APP |

Now, we will do similar transformations on the ``view_aggdf``. The ``view_aggdf`` contains information regarding the items, sessions etc.

In [4]:
# 10. Encode the device type
encoder_2 = LabelEncoder()
dt_encoder = encoder_2.fit(view_aggdf["device_type"])

# serializing and dumping the encoder for future use (for encoding the testset)
with open(os.path.join(root_dir, "models", "dt_encoder"), 'wb') as f:
    pickle.dump(dt_encoder, f)
    
# transforming
view_aggdf["device_type"] = dt_encoder.transform(view_aggdf["device_type"])

In [5]:
view_aggdf.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,0,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,0,876311,35913,15434,6656.0,9.0,9.0,244.0,9008.0
2,2018-10-15 08:58:00,0,585539,39617,22859,2528.0,13.0,67.0,170.0,2874.0
3,2018-10-15 08:58:00,0,264366,34712,11871,2547.0,4.0,38.0,62.0,10412.0
4,2018-10-15 08:58:00,0,941129,31754,127900,3520.0,1.0,28.0,283.0,908.0


In [6]:
# 11. Merging view_aggdf and train_df
inst_count_list = []            # saves user activity till a certain point in time
unique_session_id_list = []     # saves number of sessions, user is involved in till now
unique_item_id_list = []        # saves number of unique items, user has viewed till this point in time
unique_category_1_list = []     # unique category 1 viewed
unique_category_2_list = []     # unique category 2 viewed
unique_category_3_list = []     # unique category 3 viewed
unique_product_type_list = []   # unique product type viewed
item_id_mode_list = []          # captures mode of item_id till a point in time
category_1_mode_list = []       # captures mode of category_1 till a point in time
category_2_mode_list = []       # captures mode of category_2 till a point in time
category_3_mode_list = []       # captures mode of category_3 till a point in time
product_type_mode_list = []     # captures mode of product_type till a point in time
server_time_diff_mean_list = [] # captures mean of server time difference
server_time_diff_max_list = []  # captures max of server time difference
server_time_diff_min_list = []  # captures min of server time difference

for uid, imp_time in tqdm(zip(train_df["user_id"].values, train_df["impression_time"].values)):
    temp_df = view_aggdf[(view_aggdf["server_time"] < imp_time)&(view_aggdf["user_id"] == uid)]
    inst_count_list += [temp_df.shape[0]]
    
    if temp_df.shape[0] == 0:
        unique_session_id_list += [0]
        unique_item_id_list += [0]
        unique_category_1_list += [0]
        unique_category_2_list += [0]
        unique_category_3_list += [0]
        unique_product_type_list += [0]
        item_id_mode_list += [-1]
        category_1_mode_list += [-1]
        category_2_mode_list += [-1]
        category_3_mode_list += [-1]
        product_type_mode_list += [-1]
    
    else:
        unique_session_id_list += [temp_df["session_id"].nunique()]
        unique_item_id_list += [temp_df["item_id"].nunique()]
        unique_category_1_list += [temp_df["category_1"].nunique()]
        unique_category_2_list += [temp_df["category_2"].nunique()]
        unique_category_3_list += [temp_df["category_3"].nunique()]
        unique_product_type_list += [temp_df["product_type"].nunique()]
        
        if len(temp_df["item_id"].value_counts()) > 0:
            item_id_mode_list += [temp_df["item_id"].value_counts().index[0]]
        else:
            item_id_mode_list += [-1]
        
        if len(temp_df["category_1"].value_counts()) > 0:
            category_1_mode_list += [temp_df["category_1"].value_counts().index[0]]
        else:
            category_1_mode_list += [-1]
        
        if len(temp_df["category_2"].value_counts()) > 0:
            category_2_mode_list += [temp_df["category_2"].value_counts().index[0]]
        else:
            category_2_mode_list += [-1]
        
        if len(temp_df["category_3"].value_counts()) > 0:
            category_3_mode_list += [temp_df["category_3"].value_counts().index[0]]
        else:
            category_3_mode_list += [-1]
        
        if len(temp_df["product_type"].value_counts()) > 0:
            product_type_mode_list += [temp_df["product_type"].value_counts().index[0]]
        else:
            product_type_mode_list += [-1]

    server_time_diff_mean_list += [temp_df["server_time"].diff().mean()]
    server_time_diff_max_list += [temp_df["server_time"].diff().max()]
    server_time_diff_mean_list += [temp_df["server_time"].diff().min()]

train_df["inst_count"] = inst_count_list
train_df["user_unique_sessions"] = unique_session_id_list
train_df["user_unique_item_ids"] = unique_item_id_list
train_df["user_unique_category_1"] = unique_category_1_list
train_df["user_unique_category_2"] = unique_category_2_list
train_df["user_unique_category_3"] = unique_category_3_list
train_df["user_unique_product_type"] = unique_product_type_list
train_df["user_item_id_mode"] = item_id_mode_list
train_df["user_category_1_mode"] = category_1_mode_list
train_df["user_category_2_mode"] = category_2_mode_list
train_df["user_category_3_mode"] = category_3_mode_list
train_df["user_product_type_mode"] = product_type_mode_list

"""train_df["server_time_diff_mean"] = server_time_diff_mean_list
train_df["server_time_diff_max"] = server_time_diff_max_list
train_df["server_time_diff_min"] = server_time_diff_min_list

# converting the server_time derived functions to seconds
train_df["server_time_diff_mean"] = train_df["server_time_diff_mean"]/np.timedelta64(1, 's')
train_df["server_time_diff_max"] = train_df["server_time_diff_max"]/np.timedelta64(1, 's')
train_df["server_time_diff_min"] = train_df["server_time_diff_min"]/np.timedelta64(1, 's')"""

237609it [1:06:29, 59.55it/s]


'train_df["server_time_diff_mean"] = server_time_diff_mean_list\ntrain_df["server_time_diff_max"] = server_time_diff_max_list\ntrain_df["server_time_diff_min"] = server_time_diff_min_list\n\n# converting the server_time derived functions to seconds\ntrain_df["server_time_diff_mean"] = train_df["server_time_diff_mean"]/np.timedelta64(1, \'s\')\ntrain_df["server_time_diff_max"] = train_df["server_time_diff_max"]/np.timedelta64(1, \'s\')\ntrain_df["server_time_diff_min"] = train_df["server_time_diff_min"]/np.timedelta64(1, \'s\')'

In [7]:
train_df.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,diff_time_mean,diff_time_max,diff_time_min,...,user_unique_item_ids,user_unique_category_1,user_unique_category_2,user_unique_category_3,user_unique_product_type,user_item_id_mode,user_category_1_mode,user_category_2_mode,user_category_3_mode,user_product_type_mode
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,2,0,0,74670.000000,148200.0,1140.0,...,1,1,1,1,1,43886,11.0,35.0,20.0,5622.0
1,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,89464,129,0,0,0,101901.818182,347520.0,180.0,...,150,15,50,90,132,38517,17.0,9.0,62.0,8028.0
2,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,58442,127,1,0,0,68812.727273,165720.0,8400.0,...,12,8,10,10,12,73224,1.0,64.0,263.0,5164.0
3,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,4238,371,1,0,0,540.000000,540.0,540.0,...,52,12,28,40,50,37336,17.0,8.0,84.0,231.0
4,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,1,1,1,45745.882353,167340.0,360.0,...,3,2,2,2,3,43209,4.0,74.0,292.0,577.0


In [8]:
train_df.to_csv(os.path.join(root_dir, "data", "processed_data", "train_aggdf.csv"), index = False)